In [ ]:
# default_exp pubmed

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export

from datetime import datetime
from typing import List, Dict

from pydash import get
from toolz import thread_first
from typeguard import typechecked

from proseflow.spec import *
from proseflow.utils import get_paths, tree_select_kv
from Bio import Entrez

from IPython.display import JSON

from io import StringIO
from typing import List, Dict

from Bio import Entrez, Medline
from typeguard import typechecked


In [ ]:
#export
@typechecked
def _get_pubmed_records(
    pmids: List[str],
    shape=None,
):  # -> List[Tuple[str, str]]:
    # ! TODO: move this into auto convert
#     pmids_to_fetch = (
#         [pmid.split("/")[-2] for pmid in pmids]
#         if SPEC[PUBMED_IDS].match(pmids[0])
#         else pmids
#     )
    Entrez.email = "strasser.ms@gmail.com"

    # handle type is http.client.HTTPResponse
    handle = Entrez.efetch(
        db="pubmed",
        id=",".join(map(str, pmids)),
        rettype="xml",
        retmode="text",
    )
    records = Entrez.read(handle)
    return records


def get_info(record, keys_wanted):
    return thread_first(
        record,
        # (id_with_side, JSON),
        get_paths,
        list,
        # print,
        lambda paths: tree_select_kv(record, paths, keys_wanted),
        m_parse_flat_pubmed,
    )


def _get_pubmed_info(
    pmids: List[str],
    keys_wanted=[
        "PMID",
        "DateCompleted",
        "Journal",
        "PubDate",
        "AbstractText",
        "ChemicalList",
    ],
):
    records = _get_pubmed_records(pmids)
    return [get_info(r, keys_wanted) for r in records["PubmedArticle"]]


def _get_pubmed_abstracts(pmids: List[str]):
    abst = []
    for record in _get_pubmed_records(pmids)["PubmedArticle"]:
        try:
            abst += [str(get(record, "MedlineCitation.Article.Abstract.AbstractText.0"))]
        except:
            abst += [""]
    return abst


"""
Achtung:

PubDate has form:
{
"Year":"1999",
"Month":"Nov", #stupid....
"Day":"09"
}
and DateCompleted has form:
{
"Year":"1999",
"Month":"12",
"Day":"09"
}

"""

MONTH_MAP = {
    "JAN": 1,
    "FEB": 2,
    "MAR": 3,
    "APR": 4,
    "MAY": 5,
    "JUN": 6,
    "JUL": 7,
    "AUG": 8,
    "SEP": 9,
    "OCT": 10,
    "NOV": 11,
    "DEC": 12,
}


def m_parse_flat_pubmed(flat_pm):
    """! make sure that all the key_wanted are represented here too ... otherwise key error"""
    # [{a:, b:}]
    if flat_pm.get("ChemicalList"):  # only
        # stringelement
        flat_pm["ChemicalList"] = [
            str(chem["NameOfSubstance"]) for chem in flat_pm["ChemicalList"]
        ]

    if flat_pm.get("MeshHeadingList"):
        flat_pm["MeshHeadingList"] = [
            mesh["DescriptorName"] for mesh in flat_pm["MeshHeadingList"]
        ]

    if flat_pm.get("DateCompleted"):
        date_comp = {
            key.lower(): int(num)
            for key, num in flat_pm["DateCompleted"].items()
            if key
        }
        flat_pm["DateCompleted"] = str(datetime(**date_comp).date())

    # TODO: not working with month --> int("Nov") errror
    # date_pub = {key.lower(): int(num) for key, num in flat_pm["PubDate"].items()}
    # date_pub["month"] = MONTH_MAP[date_pub["month"]] #Nov => 11
    # flat_pm["PubDate"] = datetime(**date_pub).date()

    # {"DAY":"11"} -> {"day": 11}

    if flat_pm.get("AbstractText"):
        flat_pm["AbstractText"] = flat_pm["AbstractText"][0]
    if flat_pm.get("JournalTitle"):
        flat_pm["JournalTitle"] = flat_pm["Journal"]["Title"]
    if flat_pm.get("PMID"):
        flat_pm["PMID"] = flat_pm["PMID"][0:]
    if flat_pm.get("Journal"):
        del flat_pm["Journal"]

    return flat_pm


In [ ]:
JSON(_get_pubmed_records(["28355534", "33568199"])["PubmedArticle"][0])

<IPython.core.display.JSON object>

In [ ]:
[] + ["as"] + ["asa"]

['as', 'asa']

In [ ]:
#%%timeit
pm_record =_get_pubmed_abstracts(["28355534", "33568199"]) #2019: 28355534 #1999: 10544064


In [ ]:
pm_record

["Synthetic biology was founded as a biophysical discipline that sought explanations for the origins of life from chemical and physical first principles. Modern synthetic biology has been reinvented as an engineering discipline to design new organisms as well as to better understand fundamental biological mechanisms. However, success is still largely limited to the laboratory and transformative applications of synthetic biology are still in their infancy. Here, we review six principles of living systems and how they compare and contrast with engineered systems. We cite specific examples from the synthetic biology literature that illustrate these principles and speculate on their implications for further study. To fully realize the promise of synthetic biology, we must be aware of life's unique properties.",
 'Transcription factor B cell lymphoma 6 (BCL6) is a master regulator of T follicular helper (Tfh) cells, which play a crucial role in the pathogenesis of systemic lupus erythematos

In [ ]:
#JSON(pm_record)

In [ ]:
Entrez.email = 'strasser.ms@gmail.com'

In [ ]:
#export
@typechecked
def search_medline(query: str, retmax: int =100):
    ''' Searches Pubmed for <query> and returns all entries '''
    search = Entrez.esearch(db='pubmed', 
                            retmax=retmax,
                            term=query, 
                            usehistory='y')
    handle = Entrez.read(search)
    try:
        return handle
    except Exception as e:
        raise IOError(str(e))
    finally:
        search.close()

@typechecked
def fetch_details(rec_id, 
                  entrez_handle) -> str:
    ''' Fetches the details for a given publication ID '''
    fetch_handle = Entrez.efetch(db='pubmed', id=rec_id,
                                 rettype='Medline', retmode='text',
                                 webenv=entrez_handle['WebEnv'],
                                 query_key=entrez_handle['QueryKey'])
    rec = fetch_handle.read()
    return rec

@typechecked
def handle_query(query: str, 
                 records_to_retrieve: set, 
                 retrieve_max: int = 100) -> List[Dict]:
    ''' 
    Takes a query, searches for the relevant papers and returns the abstracts 
    Abbreviations:
    AB = Abstract, AID = Article Identifier, (F)AU = (Full) Author, DP = Date of Publication, 
    JT = Journal Title, OT = Other Term, PMID = PubMed Unique Identifier, TI = Title
    https://biopython.org/docs/1.75/api/Bio.Medline.html#Bio.Medline.Record
    '''
    rec_handler = search_medline(query=query,
                                 retmax=retrieve_max)

    records = []
    for rec_id in rec_handler['IdList']:
        rec = fetch_details(rec_id=rec_id, 
                            entrez_handle=rec_handler)
        rec_file = StringIO(rec)
        medline_rec = Medline.read(rec_file)
        record = {}
        for key, value in medline_rec.items():
            if key in records_to_retrieve:
                record[key] = value
        records.append(record)
        # if 'AB' in medline_rec:
        #     print(medline_rec['AB'])]
    
    return records


In [ ]:
#%%timeit
records_to_retrieve = {"AB", "AID", "AU", "DP", "JT", "OT", "PMID", "TI"}

#15 minutes for 1000 entries
records = handle_query(query='cancer immunology', 
             records_to_retrieve=records_to_retrieve,
             retrieve_max=5)


In [ ]:

from toolz import thread_first


# parsed = thread_first(
#     pm_record,
#    # (id_with_side, JSON),
#     get_paths,
#     list,
#     #print,
#    lambda paths: get_kv_from_nested(pm_record, paths,["PMID", "DateCompleted", "Journal", "PubDate", "AbstractText", "ChemicalList" ]),
#    m_parse_flat_pm
   
# )
# JSON(parsed)